<a href="https://colab.research.google.com/github/DaviAmerico/project1/blob/main/Competi%C3%A7%C3%A3o_House_Prices.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resolução da competição House Prices - Advanced Regression Techniques
Nosso objetivo aqui é criar um modelo que preveja os valores de determinadas casas com base em 80 variáveis explicativas, iremos fazer isso usando um modelo de regressão.A resolução dessa competição se dará em três tópicos: **Limpeza dos dados**,**Transformando os dados** e **Criando o modelo e prevendo os preços**.



**Limpeza dos dados**:




Nessa seção iremos tratar os dados faltantes (*nan's*), mas antes vamos importar os dados a serem usados:


In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from scipy import stats
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
#importando dados para um pandas dataframe 
train=pd.read_csv('/content/drive/MyDrive/train.csv')
test=pd.read_csv('/content/drive/MyDrive/test.csv')

O conjunto de dados *train* se refere aos dados a serem usados para a construção do modelo, já o conjunto *test* é aonde iremos fazer as previsões.

Antes de analisarmos a existência de *nan's* vamos dar uma olhada nos dois conjuntos de dados.

In [7]:
print(train.head())
print(test.head())

   Id  MSSubClass MSZoning  ...  SaleType  SaleCondition SalePrice
0   1          60       RL  ...        WD         Normal    208500
1   2          20       RL  ...        WD         Normal    181500
2   3          60       RL  ...        WD         Normal    223500
3   4          70       RL  ...        WD        Abnorml    140000
4   5          60       RL  ...        WD         Normal    250000

[5 rows x 81 columns]
     Id  MSSubClass MSZoning  ...  YrSold  SaleType SaleCondition
0  1461          20       RH  ...    2010        WD        Normal
1  1462          20       RL  ...    2010        WD        Normal
2  1463          60       RL  ...    2010        WD        Normal
3  1464          60       RL  ...    2010        WD        Normal
4  1465         120       RL  ...    2010        WD        Normal

[5 rows x 80 columns]


Percebe-se que *test* não tem os preços de venda das casas (*SalePrice*) , pois como já dito , usaremos esse conjunto para realizar as previsões,uma vez que criarmos o modelo.



Agora vamos verificar a existência de valores faltantes:

In [8]:
print(train.isna().sum().sum())
print(test.isna().sum().sum())

6965
7000


No conjunto *train* temos 6965 valores faltantes e no conjunto *test* temos 7000.Inicialmente vamos tratar os *nan's* nas colunas númericas:

In [9]:
train.fillna(train.mean(),inplace=True)
test.fillna(test.mean(),inplace=True)

Acima substituimos os valores faltantes das colunas númericas pela média dos valores em cada coluna.Pelo fato de termos usado a média as colunas categóricas não foram alteradas.

Agora vamos tratar os *nan's* de colunas categóricas, infelizmente isso não é tão simples quanto para os *nan's* númericos ,portanto teremos que usar uma recursividade para substituí-los:

In [10]:
for i in train.select_dtypes('object').columns:
    train.loc[:,i].fillna(train.loc[:,i].value_counts().index[0],inplace=True)
    test.loc[:,i].fillna(test.loc[:,i].value_counts().index[0],inplace=True)

Acima estamos percorrendo *i* nas colunas categóricas dos dados , para cada coluna os *nan's* serão substituidos pela categoria mais frequente da respectiva coluna.

Podemos ver então que não existem mais valores faltantes nos dois conjuntos de dados:

In [11]:
print(train.isna().sum().sum())
print(test.isna().sum().sum())

0
0


**Transformando os dados:**

A maioria dos algoritmos de aprendizado de maquina só aceitam dados númericos e funcionam melhor com dados normalizados.Abaixo vamos transformar todas categorias em números:

In [12]:
for i in train.select_dtypes('object').columns:
    train.loc[:,i]=LabelEncoder().fit_transform(train.loc[:,i].astype('str'))
    test.loc[:,i]=LabelEncoder().fit_transform(test.loc[:,i].astype('str'))

Podemos ver agora que cada coluna dos dois conjuntos é númerica:

In [13]:
print(train.dtypes,"\n","\n","\n")
print(test.dtypes)

Id                 int64
MSSubClass         int64
MSZoning           int64
LotFrontage      float64
LotArea            int64
                  ...   
MoSold             int64
YrSold             int64
SaleType           int64
SaleCondition      int64
SalePrice          int64
Length: 81, dtype: object 
 
 

Id                 int64
MSSubClass         int64
MSZoning           int64
LotFrontage      float64
LotArea            int64
                  ...   
MiscVal            int64
MoSold             int64
YrSold             int64
SaleType           int64
SaleCondition      int64
Length: 80, dtype: object


Antes de normalizar os dados vamos separar a variável resposta das explicativas , pois estamos interessados em apenas normalizar as explicativas:

In [14]:
xt=train.iloc[:,0:80]
yt=train.iloc[:,80]
xte=test


Em *xt* e *xte* temos as variáveis explicativas a serem normalizadas nos conjuntos *train* e *test*, note que *xte* é igual a *test*,pois, o conjunto de dados *test* não contém observações da variável resposta.

Normalizando os dados, exceto a resposta, temos:

In [15]:
zt=pd.DataFrame(StandardScaler().fit_transform(xt))
zt.columns=xt.columns
zte=pd.DataFrame(StandardScaler().fit_transform(xte))
zte.columns=xte.columns

*zt* e *zte* são os preditores normalizados para ambos os conjuntos de dados.

Agora estamos preparados para criar nosso modelo e realizar as predições :D

**Criando o modelo e prevendo os preços:**

O modelo a ser usado para resolver o problema de regressão é o *XGBoosting  (Extreme Gradient Boosting)*, ele será treinado em *(zt,yt)*.Aqui iremos testar várias combinações aleatórias de hiper-parâmetros no modelo e pegar a combinação ótima depois de um determinado número de tentativas, no caso são 25 tentativas, para nós a combinação ótima é aquela que obteve a menor raiz do erro quadrático logarítmico médio,métrica de avaliação usada na competição, dentro da validação cruzada com 5 *folds*, então:

In [ ]:
xgbr=xgb.XGBRegressor()
param_grid = {'n_estimators': stats.randint(150, 500),
              'learning_rate': stats.uniform(0.01, 0.07),
              'subsample': stats.uniform(0.3, 0.7),
              'max_depth': [3, 4, 5, 6, 7, 8, 9],
              'colsample_bytree': stats.uniform(0.45,0.5),
              'min_child_weight': [1, 2, 3]
             }
f=RandomizedSearchCV(xgbr,param_grid,cv=5,n_iter=25,scoring='neg_mean_squared_log_error')
f.fit(zt,yt)

Podemos, também, verificar o erro da melhor combinação de hiper-parâmetros:

In [ ]:
bs=np.sqrt(-f.best_score_)
print(bs)

0.12426714375527574


Agora podemos fazer a previsão dos preços das casas dentro do conjunto *test* normalizado:

In [ ]:
yp=f.best_estimator_.predict(zte)
print(yp)

[129399.016 153070.3   187078.2   ... 174546.45  121318.96  250490.44 ]


Agora basta exportar as predições para o Kaggle para concluir a competição :D